In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler

In [ ]:
# Upload your CSV from local
from google.colab import files
uploaded = files.upload()

# Load it into a DataFrame
import pandas as pd
df=pd.read_excel("survey.xlsx")



In [ ]:
df.head()

In [ ]:
# List the columns you want to drop
columns_to_drop = ['survey_id', 'survey_year', 'end_date']  # Replace with actual column names

# Drop the columns
df = df.drop(columns=columns_to_drop)

In [ ]:
df = df.drop(columns='combination_type')

In [ ]:
df = df[df['answer_value'] != 12]

In [ ]:
df_883 = df[df['question_id'] == 883]

In [ ]:
df_883 = df_883.drop(columns='answer_value')

In [ ]:
df_883 = df_883.drop(columns='result_code')

In [ ]:
df_883 = df_883.drop(columns='question_id')

In [ ]:
df_883 = df_883.drop(columns='question_text')

In [ ]:
df_883.head()

In [ ]:
df_869 = df[df['question_id'] == 869]

In [ ]:
df_869 = df_869[['result_id', 'answer_value']]

In [ ]:
df_869 = df_869[df_869['answer_value'] != 6]

In [ ]:
df_869.head()

,result_id,answer_value
1,116234,1
3,116235,2
5,116236,1
7,116237,1
9,116238,1


In [ ]:
df=pd.merge(df_883, df_869,on="result_id", how="inner")

In [ ]:
df.head()

In [ ]:
df.isnull().sum().sort_values(ascending=False)

In [ ]:
counts = df['result_id'].nunique()

In [ ]:
customer_names = df['customer_name'].unique()

In [ ]:
lista=["answer_value"]

In [ ]:
summary_data = []

for k in customer_names:
    customer_df = df[df['customer_name'] == k]
    df_top = customer_df[["customer_name", "combination_value", "answer_value"]].copy()
    x = "answer_value"
    if k == 'xy':
        user_groups = customer_df['usergroup'].unique()
        for idx, m in enumerate(user_groups, start=1):
          user_df = customer_df[customer_df['usergroup'] == m]

          # Rename
          new_name = f"{k} - AD {idx}"
          mask = (customer_df['customer_name'] == 'xy') & (customer_df['usergroup'] == m)
          customer_df.loc[mask, 'customer_name'] = new_name

          for z in range(1, 6):
              df_top = user_df[user_df[x] == z].copy()
              df_top['NPS_Category'] = df_top['combination_value']
              promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
              detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
              total = len(df_top)
              passives = total - promoters - detractors

              summary_data.append({
                  'Customer Name': new_name,
                  'User Group': f"AD {idx}",
                  'Value': z,
                  'Total Responses': total,
                  'Promoters': promoters,
                  'Passives': passives,
                  'Detractors': detractors,
              })

          # ✅ Now add the "Insgesamt" row per AD group
          df_all_ad = user_df.copy()
          df_all_ad['NPS_Category'] = df_all_ad['combination_value']
          promoters = len(df_all_ad[df_all_ad['NPS_Category'] == 'Promoter'])
          detractors = len(df_all_ad[df_all_ad['NPS_Category'] == 'Detractor'])
          total = len(df_all_ad)
          passives = total - promoters - detractors

          summary_data.append({
              'Customer Name': new_name,
              'User Group': f"AD {idx}",
              'Value': 'Insgesamt',
              'Total Responses': total,
              'Promoters': promoters,
              'Passives': passives,
              'Detractors': detractors
          })

    else:
        for z in range(1, 6):
            df_top = customer_df[customer_df[x] == z].copy()
            df_top['NPS_Category'] = df_top['combination_value']
            promoters = len(df_top[df_top['NPS_Category'] == 'Promoter'])
            detractors = len(df_top[df_top['NPS_Category'] == 'Detractor'])
            total = len(df_top)
            passives=total-promoters-detractors

            summary_data.append({
                'Customer Name': k,
                'User Group': 'All',
                'Value': z,
                'Total Responses': total,
                'Promoters': promoters,
                'Passives' : passives,
                'Detractors': detractors,
            })

        df_all = customer_df.copy()
        df_all['NPS_Category'] = df_all['combination_value']
        promoters = len(df_all[df_all['NPS_Category'] == 'Promoter'])
        detractors = len(df_all[df_all['NPS_Category'] == 'Detractor'])
        total = len(df_all)

        summary_data.append({
            'Customer Name': k,
            'User Group': 'All',
            'Value': 'Insgesamt',
            'Total Responses': total,
            'Promoters': promoters,
            'Passives' : passives,
            'Detractors': detractors
        })

# === Create summary DataFrame ===
summary_df = pd.DataFrame(summary_data)

# === Weighting logic ===
unique_customers = summary_df['Customer Name'].nunique()
weight_base = counts / unique_customers


summary_df['Weight_Base'] = weight_base

customer_totals = summary_df[summary_df['Value'] == 'Insgesamt'].set_index('Customer Name')['Total Responses'].to_dict()

summary_df['Weight_Company'] = summary_df['Customer Name'].map(customer_totals).apply(
    lambda x: weight_base / x if x > 0 else None
)

summary_df['Weighted_total'] = summary_df['Total Responses'] * summary_df['Weight_Company']
summary_df['Weighted_Promoters'] = summary_df['Promoters'] * summary_df['Weight_Company']
summary_df['Weighted_Detractors'] = summary_df['Detractors'] * summary_df['Weight_Company']
summary_df['Weighted_Passives'] = summary_df['Passives'] * summary_df['Weight_Company']

In [ ]:
summary_df


In [ ]:
import pandas as pd

# Step 1: Create an empty DataFrame with two columns
df_final = pd.DataFrame(columns=['NAME', 'PROMOTORS'])

# Step 2: Add rows in a loop
for i in summary_df:  # Change the range as needed
    value1 = f"Value{i}"
    value2 = i * 10
    df_final.loc[len(df)] = [value1, value2]  # Append a new row

# Step 3: Display the DataFrame
df_final.head()

In [ ]:
import pandas as pd

# Group by 'Value' and sum both 'Weighted_Promoters' and 'Weighted_Detractors'
df_final = summary_df.groupby('Value', as_index=False)[['Weighted_Promoters', 'Weighted_Passives', 'Weighted_Detractors', 'Weighted_total']].sum()

# Optional: rename columns for clarity
df_final.rename(columns={
    'Weighted_Promoters': 'Sum_Weighted_Promoters',
    'Weighted_Detractors': 'Sum_Weighted_Detractors',
    'Weighted_Passives': 'Sum_Weighted_Passives'
}, inplace=True)

In [ ]:
df_final["pos_pro"] = ((df_final["Sum_Weighted_Promoters"] / df_final["Weighted_total"])) * 100
df_final['pos_pas'] = ((df_final['Sum_Weighted_Passives'] / df_final["Weighted_total"])) * 100
df_final["pos_det"] = (df_final["Sum_Weighted_Detractors"] / df_final["Weighted_total"]) * 100


In [ ]:
df_final["nps"]=df_final["pos_pro"]-df_final["pos_det"]

In [ ]:
df_final